In [1]:
import pandas as pd
import numpy as np

file_path = 'data/rhDNase.xlsx'

data_df = pd.read_excel(file_path)
data_df.head()

,id,trt,fev,fev2,start,stop,event,etype,enum,enum1,emum2
0,493301,1,28.8,28.1,0,168,0,1,1,1,0
1,493303,1,64.0,63.0,0,169,0,1,1,1,0
2,493305,0,67.2,68.7,0,65,1,1,1,1,0
3,493305,0,67.2,68.7,65,75,1,2,2,1,1
4,493305,0,67.2,68.7,75,168,0,1,3,2,1


In [2]:
data_df.describe()

,id,trt,fev,fev2,start,stop,event,etype,enum,enum1,emum2
count,1.326000e+03,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000,1326.000000
mean,9.325327e+05,0.467572,56.115837,56.058296,46.843137,127.645551,0.543741,1.272247,2.177225,1.447210,0.730015
std,1.025395e+05,0.499136,24.912700,24.886098,55.469205,50.480106,0.498271,0.445284,1.551956,0.746213,0.868460
min,4.933010e+05,0.000000,16.000000,17.300000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,9.193140e+05,0.000000,35.200000,35.625000,0.000000,88.000000,0.000000,1.000000,1.000000,1.000000,0.000000
50%,9.743050e+05,0.000000,51.200000,51.850000,13.000000,157.000000,1.000000,1.000000,2.000000,1.000000,1.000000
75%,9.873195e+05,1.000000,73.600000,73.100000,95.000000,169.000000,1.000000,2.000000,3.000000,2.000000,1.000000
max,1.024313e+06,1.000000,140.800000,141.500000,170.000000,196.000000,1.000000,2.000000,10.000000,5.000000,5.000000
